# データ可視化ツールの発展

## 準備

In [40]:
# warningsモジュールのインポート
import warnings

# データ解析や機械学習のライブラリ使用時の警告を非表示にする目的で警告を無視
# 本書の文脈では、可視化の学習に議論を集中させるために選択した
# ただし、学習以外の場面で、警告を無視する設定は推奨しない
warnings.filterwarnings("ignore")

In [41]:
# polarsとrayをインストール
!pip install polars ray

In [42]:
# sysモジュールのインポート
# Pythonインタプリタに関する情報や操作を提供
import sys

# numbaモジュールのインポート
# Pythonコードを高速化するためのJITコンパイラ
import numba

# numpyモジュールのインポート
# 数値計算やデータ分析に特化した多次元配列を提供
# npという名前で参照可能
import numpy as np

# pandasモジュールのインポート
# データ解析ライブラリ
# pdという名前で参照可能
import pandas as pd

# polarsモジュールのインポート
# 高速なデータ処理ライブラリ
# plという名前で参照可能
import polars as pl

# rayモジュールのインポート
# 分散コンピューティングフレームワーク
# 大規模なデータ処理や機械学習のタスクを並列化
import ray

## Python

In [43]:
# pythonのバージョンを表示
sys.version

'3.11.6 | packaged by conda-forge | (main, Oct  3 2023, 11:57:02) [GCC 12.3.0]'

In [44]:
# numpyのバージョンを確認
np.__version__

'1.24.4'

In [45]:
# numbaのバージョンを確認
numba.__version__

'0.57.1'

In [46]:
def sum_python(arr):
    # 結果を格納するための変数を初期化
    result = 0

    # リストの要素を順番に足していく
    for x in arr:
        result += x

    # 結果を返す
    return result


@numba.jit(nopython=True)
def sum_numba(arr):
    # 結果を格納するための変数を初期化
    result = 0

    # リストの要素を順番に足していく（Numbaでコンパイル）
    for x in arr:
        result += x

    # 結果を返す
    return result


# 大きな配列をランダムに生成
arr = np.random.rand(10000000)

In [47]:
# 通常のPythonの実行速度を計測
%timeit sum_python(arr)

1.17 s ± 10.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [48]:
# NumbaのJITコンパイルを用いた実行速度を計測
%timeit sum_numba(arr)

9.6 ms ± 75.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [49]:
# rayのバージョンを確認
ray.__version__

'2.37.0'

In [50]:
@ray.remote
def process_chunk(chunk):
    # 各チャンクに対する処理を記述
    # この例では、チャンクの合計値を計算
    return np.sum(chunk)

In [51]:
# ローカルで実行可能な規模のダミーデータを生成
big_data = np.random.rand(1_000_000)

In [52]:
# 大規模データを100個のチャンクに分割
# np.array_splitを使用して、大規模データを等分割
chunks = np.array_split(big_data, 100)

# 各チャンクを並列に処理
# process_chunk関数をリモート実行するタスクを作成
# .remote()を使用して、タスクをRayクラスタに送信
futures = [process_chunk.remote(chunk) for chunk in chunks]

# 結果を集約
# ray.getを使用して、並列実行された各タスクの結果を取得
result = ray.get(futures)

In [53]:
# 検算用に、チャンクに分けずに合算
sum(result), sum(big_data)

(500192.7159108848, 500192.71591088624)

## Pandas

In [54]:
# ダミーデータの設定
num_rows = 1000000
num_cols = 3

# ランダムなデータを生成
np.random.seed(0)
data = {
    "col1": np.random.rand(num_rows).astype(np.float32),
    "col2": np.random.randint(0, 100, num_rows, dtype=np.int8),
    "col3": np.random.choice(["A", "B", "C"], num_rows),
}

# DataFrameの作成
df = pd.DataFrame(data)

# CSVファイルに保存
file_path = "../../../data/sandbox/large_data.csv"
df.to_csv(file_path, index=False)

In [55]:
# 特に何も指定せずに読み込む
df_naive = pd.read_csv(file_path)
# メモリ使用量等の情報を表示
print(df_naive.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 3 columns):
 #   Column  Non-Null Count    Dtype  
---  ------  --------------    -----  
 0   col1    1000000 non-null  float64
 1   col2    1000000 non-null  int64  
 2   col3    1000000 non-null  object 
dtypes: float64(1), int64(1), object(1)
memory usage: 22.9+ MB
None


In [56]:
# dtypeを指定して読み込む
df_dtype = pd.read_csv(
    file_path, dtype={"col1": "float32", "col2": "int8", "col3": "category"}
)
# メモリ消費量等の情報を表示
print(df_dtype.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 3 columns):
 #   Column  Non-Null Count    Dtype   
---  ------  --------------    -----   
 0   col1    1000000 non-null  float32 
 1   col2    1000000 non-null  int8    
 2   col3    1000000 non-null  category
dtypes: category(1), float32(1), int8(1)
memory usage: 5.7 MB
None


In [57]:
# チャンク一つあたりのサイズを指定
chunk_size = 100000

# chunksize引数を指定してread_csv()し、チャンクごとに処理
for chunk in pd.read_csv(file_path, chunksize=chunk_size):
    # 各チャンクに対する処理を記述
    # ここでは単純にチャンクのサイズを表示
    print(f"Processing chunk of size {len(chunk)}")


Processing chunk of size 100000
Processing chunk of size 100000
Processing chunk of size 100000
Processing chunk of size 100000
Processing chunk of size 100000
Processing chunk of size 100000
Processing chunk of size 100000
Processing chunk of size 100000
Processing chunk of size 100000
Processing chunk of size 100000


In [58]:
# 1000万行 × 10列のデータを生成
data = np.random.rand(10000000, 10)
df_pandas = pd.DataFrame(data)
df_polars = pl.DataFrame(data)

In [59]:
# pandasの処理時間を計測
%timeit df_pandas.sum()

263 ms ± 6.49 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [60]:
# polarsの処理時間を計測
%timeit df_polars.sum()

8.16 ms ± 153 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
